In [1]:
# Cell 1: Import and setup
import re
import json
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from tqdm import tqdm
import google.generativeai as genai
import time
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import RealDictCursor, Json
import logging
from typing import Dict, List, Optional

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


c:\Users\malac\anaconda3\envs\pathfinder\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Cell 2: Configure API and Database
# Load environment variables from .env file
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
DATABASE_URL = os.getenv("DATABASE_URL")

# Set GSSAPI disable for Windows
os.environ['PGGSSENCMODE'] = 'disable'

if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not found in .env file")

if not DATABASE_URL:
    raise ValueError("DATABASE_URL not found in .env file")

genai.configure(api_key=GEMINI_API_KEY)

BASE_URL = "https://catalog.unc.edu"
PROGRAMS_URL = f"{BASE_URL}/undergraduate/programs-study/"


In [3]:
# Cell 3: Database Manager for Programs
class ProgramDatabaseManager:
    def __init__(self, db_url: str):
        """Initialize database connection for programs."""
        # Parse the URL to add gssencmode parameter
        from urllib.parse import urlparse
        url = urlparse(db_url)
        
        conn_params = {
            "host": url.hostname,
            "port": url.port,
            "database": url.path[1:],  # Remove leading '/'
            "user": url.username,
            "password": url.password,
            "sslmode": "require",
            "gssencmode": "disable"  # Fix for Windows GSSAPI error
        }
        
        self.conn = psycopg2.connect(**conn_params)
        self.conn.autocommit = False
        self.cur = self.conn.cursor(cursor_factory=RealDictCursor)
        
        # Cache for course lookups
        self.course_id_cache = {}
        self._load_course_cache()
    
    def _load_course_cache(self):
        """Load existing courses into cache for linking requirements."""
        self.cur.execute("SELECT id, course_id FROM courses")
        for row in self.cur.fetchall():
            self.course_id_cache[row['course_id']] = row['id']
        logger.info(f"Loaded {len(self.course_id_cache)} courses into cache")
    
    def save_program(self, program_data: Dict) -> Optional[int]:
        """Save a program to the database."""
        try:
            # Infer program type from name if not provided
            program_type = program_data.get('program_type')
            if not program_type:
                name_lower = program_data.get('program_name', '').lower()
                if 'minor' in name_lower:
                    program_type = 'minor'
                elif 'major' in name_lower:
                    program_type = 'major'
                elif 'certificate' in name_lower:
                    program_type = 'certificate'
                else:
                    program_type = 'major'  # Default assumption
            
            # Log what we're saving
            logger.info(f"Saving {program_data.get('program_name')} as {program_type}")
            
            # Insert or update program
            self.cur.execute("""
                INSERT INTO programs 
                (program_id, name, program_type, degree_type, total_hours, url)
                VALUES (%s, %s, %s, %s, %s, %s)
                ON CONFLICT (program_id) DO UPDATE SET
                    name = EXCLUDED.name,
                    program_type = EXCLUDED.program_type,
                    degree_type = EXCLUDED.degree_type,
                    total_hours = EXCLUDED.total_hours,
                    url = EXCLUDED.url,
                    updated_at = NOW()
                RETURNING id
            """, (
                program_data.get('program_id'),
                program_data.get('program_name'),
                program_type,  # Use inferred type
                program_data.get('degree_type'),
                program_data.get('total_hours'),
                program_data.get('url')
            ))
            
            program_db_id = self.cur.fetchone()['id']
            
            # Save requirements if present
            if program_data.get('requirements'):
                self._save_program_requirements(program_db_id, program_data['requirements'])
            
            return program_db_id
            
        except Exception as e:
            logger.error(f"Error saving program {program_data.get('program_name')}: {e}")
            raise
    
    def _save_program_requirements(self, program_db_id: int, requirements: Dict):
        """Save requirements for a program."""
        # Clear existing requirements
        self.cur.execute("""
            DELETE FROM program_requirement_courses 
            WHERE requirement_id IN (
                SELECT id FROM program_requirements WHERE program_id = %s
            )
        """, (program_db_id,))
        self.cur.execute("DELETE FROM program_requirements WHERE program_id = %s", (program_db_id,))
        
        display_order = 0
        
        # Process different requirement types
        req_type_mappings = [
            ('gateway_courses', 'gateway'),
            ('core_requirements', 'core'),
            ('electives', 'elective'),
            ('allied_sciences', 'allied_science')
        ]
        
        for json_key, req_type in req_type_mappings:
            if json_key not in requirements:
                continue
            
            items = requirements[json_key]
            if not isinstance(items, list):
                continue
            
            for item in items:
                # Determine category name and other fields based on structure
                category_name = None
                min_credits = None
                min_courses = None
                selection_notes = None
                level_requirement = None
                other_restrictions = None
                
                # Handle different item structures
                if isinstance(item, dict):
                    category_name = item.get('category', item.get('description', json_key))
                    min_credits = item.get('min_credits', item.get('total_credits', item.get('credits')))
                    min_courses = item.get('min_courses', item.get('count'))
                    selection_notes = item.get('selection_notes', item.get('notes'))
                    level_requirement = item.get('level_requirement')
                    other_restrictions = item.get('restrictions', item.get('other_restrictions'))
                
                # Insert requirement category
                self.cur.execute("""
                    INSERT INTO program_requirements
                    (program_id, requirement_type, category_name, min_credits, 
                     min_courses, selection_notes, level_requirement, 
                     other_restrictions, display_order)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                    RETURNING id
                """, (
                    program_db_id,
                    req_type,
                    category_name,
                    min_credits,
                    min_courses,
                    selection_notes,
                    level_requirement,
                    other_restrictions,
                    display_order
                ))
                
                req_id = self.cur.fetchone()['id']
                display_order += 1
                
                # Extract and save course associations
                self._save_requirement_courses(req_id, item)
    
    def _save_requirement_courses(self, req_id: int, item: Dict):
        """Save course associations for a requirement."""
        courses_to_add = []
        
        # Handle different structures for course lists
        if 'courses' in item and isinstance(item['courses'], list):
            for course in item['courses']:
                if isinstance(course, dict):
                    courses_to_add.append({
                        'course_code': course.get('course_code', course.get('course')),
                        'is_required': True
                    })
                elif isinstance(course, str):
                    courses_to_add.append({'course_code': course, 'is_required': True})
        
        # Single course entries
        elif 'course_code' in item:
            courses_to_add.append({
                'course_code': item['course_code'],
                'is_required': True
            })
        
        # Required courses list
        elif 'required_courses' in item and isinstance(item['required_courses'], list):
            for course_code in item['required_courses']:
                courses_to_add.append({'course_code': course_code, 'is_required': True})
        
        # Optional courses lists
        elif 'options' in item and isinstance(item['options'], list):
            for course_code in item['options']:
                courses_to_add.append({'course_code': course_code, 'is_required': False})
        elif 'specific_options' in item and isinstance(item['specific_options'], list):
            for course_code in item['specific_options']:
                courses_to_add.append({'course_code': course_code, 'is_required': False})
        
        # Insert course associations
        for course_info in courses_to_add:
            course_code = course_info['course_code']
            if not course_code:
                continue
                
            course_db_id = self.course_id_cache.get(course_code)
            if course_db_id:
                try:
                    self.cur.execute("""
                        INSERT INTO program_requirement_courses
                        (requirement_id, course_id, is_required)
                        VALUES (%s, %s, %s)
                        ON CONFLICT (requirement_id, course_id) DO NOTHING
                    """, (req_id, course_db_id, course_info['is_required']))
                except Exception as e:
                    logger.warning(f"Failed to link course {course_code} to requirement: {e}")
    
    def commit(self):
        """Commit the current transaction."""
        self.conn.commit()
    
    def rollback(self):
        """Rollback the current transaction."""
        self.conn.rollback()
    
    def close(self):
        """Close database connection."""
        self.cur.close()
        self.conn.close()


In [8]:
# Cell 4: RequirementsParser class
class RequirementsParser:
    def __init__(self, model="gemini-1.5-flash", delay: float = 2.1):
        """Initialize the parser with Gemini API."""
        self.model = genai.GenerativeModel(model)
        self.delay = delay
        self.api_calls = 0
        self.failed_parses = []
        self.last_call_time = 0
        
    def parse_requirements(self, html_content: str, program_name: str = None) -> dict:
        """Parse requirements from HTML content using Gemini API."""
        import time
        
        # Extract the right-col div content
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Try to find the requirements tab content
        requirements_div = soup.find('div', {'id': 'requirementstextcontainer'})
        if not requirements_div:
            # Fallback to right-col
            requirements_div = soup.find('div', {'id': 'right-col'})
        
        if not requirements_div:
            return {
                "error": "No requirements content found",
                "requirements": {}
            }
        
        # Convert tables to structured text for better parsing
        requirements_html = str(requirements_div)
        
        # Log the size of content being sent
        content_size = len(requirements_html)
        print(f"   HTML content size: {content_size} characters")
        
        # If content is too large, try to extract just the essential parts
        if content_size > 50000:  # If over 50k characters
            print("   Content too large, extracting essential parts...")
            # Extract just tables and important text
            tables = requirements_div.find_all('table')
            essential_html = ""
            for table in tables:
                essential_html += str(table)
            requirements_html = essential_html
            print(f"   Reduced to: {len(requirements_html)} characters")
        
        # Rate limiting
        current_time = time.time()
        time_since_last_call = current_time - self.last_call_time
        if time_since_last_call < self.delay:
            sleep_time = self.delay - time_since_last_call
            time.sleep(sleep_time)
        
        self.last_call_time = time.time()
        self.api_calls += 1
        
        prompt = f"""Parse the following major/minor requirements HTML and return a JSON object with this structure:

{{
    "program_type": "major" or "minor" or "certificate" (REQUIRED - infer from the program name if not explicitly stated),
    "degree_type": "BA" or "BS" or "BSPH" etc if specified,
    "total_hours": // Total credit hours required (number or null),
    "requirements": {{
        "gateway_courses": [
            {{
                "course_code": "BIOL 101",
                "course_name": "Principles of Biology",
                "credits": 4,
                "lab_code": "BIOL 101L", // if has a lab
                "notes": "any special notes"
            }}
        ],
        "core_requirements": [
            {{
                "category": "Fundamentals Core Courses", // or whatever the category is called
                "courses": [
                    {{
                        "course_code": "BIOL 103",
                        "course_name": "How Cells Function",
                        "credits": 3,
                        "notes": null
                    }}
                ],
                "total_credits": 9,
                "selection_notes": null // e.g., "choose 2 from list"
            }}
        ],
        "electives": [
            {{
                "category": "Biology electives",
                "min_credits": 12,
                "min_courses": null,
                "level_requirement": "200 or above",
                "specific_options": ["BIOL 220", "BIOL 240"], // if specific courses listed
                "restrictions": "At least one course must have a laboratory",
                "notes": null
            }}
        ],
        "allied_sciences": [
            {{
                "category": "Chemistry",
                "required_courses": ["CHEM 101", "CHEM 102"],
                "min_credits": 8,
                "notes": "With labs"
            }}
        ],
        "additional_requirements": [
            "Students must fulfill all General Education requirements",
            "Minimum 2.0 GPA in major courses"
        ]
    }},
    "footnotes": [
        // Any superscript notes from the tables
    ],
    "special_notes": // Any important notes about the program
}}

IMPORTANT: The program_type field is REQUIRED. Look at the program name being parsed:
Program name: {program_name}
- If it contains "Minor", set program_type to "minor"  
- If it contains "Major", set program_type to "major"
- If it contains "Certificate", set program_type to "certificate"
- Default to "major" if unclear

Parse the HTML carefully:
1. Look for table structures with course codes, names, and credit hours
2. Identify section headers (Gateway Course, Core Requirements, etc.)
3. Extract exact course codes (DEPT ###) and credit hours
4. Note any footnotes marked with superscripts
5. Capture selection rules (e.g., "Two of the following five Core Course options")
6. Include lab courses that are mentioned
7. Extract total hour requirements if stated
8. Note any GPA or grade requirements

HTML content to parse:
{requirements_html}

Return ONLY the JSON object, no explanation or markdown."""

        try:
            response = self.model.generate_content(prompt)
            json_text = response.text.strip()
            json_text = re.sub(r'^```json\s*', '', json_text)
            json_text = re.sub(r'\s*```$', '', json_text)
            
            result = json.loads(json_text)
            return result
            
        except Exception as e:
            if program_name:
                self.failed_parses.append((program_name, str(e)))
            return {
                "error": f"Failed to parse: {str(e)}",
                "requirements": {},
                "program_name": program_name
            }

In [9]:
# Cell 5: Scraping functions
def get_program_links():
    """Get all program links from the programs page."""
    response = requests.get(PROGRAMS_URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the az_sitemap div
    sitemap = soup.find('div', {'class': 'az_sitemap'})
    if not sitemap:
        print("Warning: Could not find az_sitemap div")
        return []
    
    links = []
    for a in sitemap.find_all('a', href=True):
        href = a['href']
        text = a.text.strip()
        
        # Skip non-program links
        if 'programs-study' not in href:
            continue
            
        # Get the full URL
        full_url = urljoin(BASE_URL, href)
        
        # Add #requirementstext to the URL to go directly to requirements
        if '#' not in full_url:
            full_url += '#requirementstext'
        
        links.append({
            'name': text,
            'url': full_url,
            'program_id': href.split('/')[-2] if '/' in href else href
        })
    
    return links

def scrape_program(url: str, parser: RequirementsParser, program_info: dict, 
                  db_manager: Optional[ProgramDatabaseManager] = None, mode: str = 'database'):
    """Scrape a single program's requirements."""
    try:
        # Time the request
        import time
        start_time = time.time()
        response = requests.get(url, timeout=30)  # Add 30 second timeout
        response.raise_for_status()
        request_time = time.time() - start_time
        print(f"   Request took: {request_time:.1f}s")
        
        # Time the parsing
        parse_start = time.time()
        result = parser.parse_requirements(response.text, program_info['name'])
        parse_time = time.time() - parse_start
        print(f"   Parsing took: {parse_time:.1f}s")
        
        # Add program metadata
        result['program_name'] = program_info['name']
        result['program_id'] = program_info['program_id']
        result['url'] = url
        
        # Save to database if in database mode
        if mode in ['database', 'both'] and db_manager and 'error' not in result:
            try:
                save_start = time.time()
                db_manager.save_program(result)
                save_time = time.time() - save_start
                print(f"   Database save took: {save_time:.1f}s")
            except Exception as e:
                logger.error(f"Failed to save program {program_info['name']}: {e}")
                result['database_error'] = str(e)
        
        return result
        
    except Exception as e:
        logger.error(f"Error scraping {program_info['name']}: {e}")
        return {
            'program_name': program_info['name'],
            'program_id': program_info['program_id'],
            'url': url,
            'error': str(e)
        }

def scrape_all_programs(parser: RequirementsParser, db_manager: Optional[ProgramDatabaseManager] = None,
                       limit: Optional[int] = None, mode: str = 'database', dry_run: bool = False):
    """
    Scrape all programs with flexible output options.
    
    Args:
        parser: RequirementsParser instance
        db_manager: ProgramDatabaseManager instance (required for database mode)
        limit: Maximum number of programs to scrape (None for all)
        mode: 'database', 'json', or 'both'
        dry_run: If True, don't actually save anything
    """
    # Get all program links
    print("🔍 Finding all program links...")
    program_links = get_program_links()
    
    if limit:
        program_links = program_links[:limit]
    
    print(f"\n🎯 Found {len(program_links)} programs to scrape")
    print(f"   Mode: {mode}")
    print(f"   Dry run: {dry_run}\n")
    
    all_programs = []
    overall_start_time = time.time()
    saved_count = 0
    
    for idx, program_info in enumerate(program_links, 1):
        print(f"📚 [{idx}/{len(program_links)}] Scraping {program_info['name']}...")
        program_start_time = time.time()
        
        # Begin transaction for this program
        if db_manager and not dry_run:
            db_manager.conn.commit()  # Commit any pending changes
        
        result = scrape_program(
            program_info['url'], 
            parser, 
            program_info,
            db_manager if not dry_run else None,
            mode
        )
        
        if mode in ['json', 'both']:
            all_programs.append(result)
        
        # Commit program transaction
        if db_manager and not dry_run and mode in ['database', 'both'] and 'error' not in result:
            try:
                db_manager.commit()
                saved_count += 1
            except Exception as e:
                logger.error(f"Failed to commit {program_info['name']}: {e}")
                db_manager.rollback()
        
        program_elapsed = time.time() - program_start_time
        status = "✅" if 'error' not in result else "⚠️"
        print(f"{status} Completed {program_info['name']} in {program_elapsed:.1f} seconds\n")
    
    overall_elapsed = time.time() - overall_start_time
    print(f"⏱️  Total scraping time: {overall_elapsed/60:.1f} minutes")
    print(f"💾 Saved {saved_count} programs to database")
    
    return all_programs

def save_to_json(data, filename="unc_programs.json"):
    """Save program data to JSON file."""
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"\n💾 Saved to {filename}")


In [10]:
# Cell 6: Analysis functions
def analyze_programs(programs):
    """Analyze the scraped program data."""
    total = len(programs)
    successful = len([p for p in programs if 'error' not in p])
    failed = total - successful
    
    print(f"\n📊 Scraping Summary:")
    print(f"   Total programs: {total}")
    print(f"   Successfully parsed: {successful}")
    print(f"   Failed to parse: {failed}")
    
    if failed > 0:
        print(f"\n⚠️  Failed programs:")
        for p in programs:
            if 'error' in p:
                print(f"   - {p['program_name']}: {p['error'][:50]}...")
    
    # Count majors vs minors
    majors = [p for p in programs if 'major' in p.get('program_name', '').lower()]
    minors = [p for p in programs if 'minor' in p.get('program_name', '').lower()]
    
    print(f"\n📈 Program Types:")
    print(f"   Majors: {len(majors)}")
    print(f"   Minors: {len(minors)}")
    print(f"   Other: {total - len(majors) - len(minors)}")


In [11]:
# Cell 7: Main execution
# Initialize components
parser = RequirementsParser(model="gemini-1.5-flash", delay=0.5)  # Reduced delay
db_manager = ProgramDatabaseManager(DATABASE_URL)

# Configuration options
MODE = 'database'  # 'database', 'json', or 'both'
DRY_RUN = False    # Set to True to test without saving
LIMIT = 5          # Set to None to scrape all programs

# Test with a few programs first
print("🧪 Testing with first 5 programs...\n")
test_programs = scrape_all_programs(
    parser,
    db_manager,
    limit=LIMIT,
    mode=MODE,
    dry_run=DRY_RUN
)

# Save JSON backup if requested
if MODE in ['json', 'both'] and test_programs:
    save_to_json(test_programs, "unc_programs_test.json")
    analyze_programs(test_programs)

# Option to scrape all programs (uncomment to use)
# print("\n🚀 Scraping all programs...\n")
# all_programs = scrape_all_programs(parser, db_manager, mode=MODE)
# if MODE in ['json', 'both'] and all_programs:
#     save_to_json(all_programs, "unc_programs_all.json")
#     analyze_programs(all_programs)

# Print API usage stats
print(f"\n📊 API Statistics:")
print(f"   Total API calls: {parser.api_calls}")
print(f"   Failed parses: {len(parser.failed_parses)}")
if parser.failed_parses:
    print(f"\n   Failed programs:")
    for name, error in parser.failed_parses[:5]:
        print(f"   - {name}: {error[:50]}...")

# Close database connection
db_manager.close()


2025-07-21 12:21:01,881 - INFO - Loaded 123 courses into cache


🧪 Testing with first 5 programs...

🔍 Finding all program links...

🎯 Found 5 programs to scrape
   Mode: database
   Dry run: False

📚 [1/5] Scraping Aerospace Studies Minor...
   Request took: 0.2s
   HTML content size: 4792 characters


2025-07-21 12:21:54,986 - INFO - Saving Aerospace Studies Minor as minor


   Parsing took: 52.6s
   Database save took: 0.6s
✅ Completed Aerospace Studies Minor in 53.4 seconds

📚 [2/5] Scraping African American and Diaspora Studies Minor...
   Request took: 0.3s
   HTML content size: 7509 characters


2025-07-21 12:22:13,046 - INFO - Saving African American and Diaspora Studies Minor as minor


...

   Parsing took: 7.3s
   Database save took: 0.4s
✅ Completed African Studies Minor in 8.0 seconds

📚 [4/5] Scraping African, African American, and Diaspora Studies Major, B.A....
   Request took: 0.2s
   HTML content size: 13667 characters


2025-07-21 12:22:30,908 - INFO - Saving African, African American, and Diaspora Studies Major, B.A. as major


   Parsing took: 8.6s
   Database save took: 1.4s
✅ Completed African, African American, and Diaspora Studies Major, B.A. in 10.4 seconds

📚 [5/5] Scraping American Indian and Indigenous Studies Minor...
   Request took: 0.3s
   HTML content size: 19072 characters


2025-07-21 12:26:53,159 - INFO - Saving American Indian and Indigenous Studies Minor as minor


   Parsing took: 260.4s
   Database save took: 0.2s
✅ Completed American Indian and Indigenous Studies Minor in 261.0 seconds

⏱️  Total scraping time: 5.9 minutes
💾 Saved 5 programs to database

📊 API Statistics:
   Total API calls: 5
   Failed parses: 0


In [12]:
# Cell 8: Database verification
def verify_program_scraping():
    """Verify what was scraped into the database."""
    from db_queries import CourseDatabase
    
    with CourseDatabase() as db:
        # Get program stats
        stats = db.get_database_stats()
        print(f"\n🔍 Program Statistics:")
        print(f"   Total programs: {stats['total']}")
        print(f"   Majors: {stats['majors']}")
        print(f"   Minors: {stats['minors']}")
        
        # Test program lookup
        programs = db.search_programs("computer science")
        if programs:
            print(f"\n📚 Found {len(programs)} Computer Science programs:")
            for prog in programs:
                print(f"   - {prog['name']} ({prog['program_type']})")
                
                # Get requirements
                reqs = db.get_program_requirements(prog['program_id'])
                print(f"     Requirements: {len(reqs)} categories")
                for req in reqs[:2]:  # Show first 2
                    print(f"       • {req['category_name']} ({req['requirement_type']})")
                    if req['courses']:
                        print(f"         Courses: {len(req['courses'])} linked")

# Run verification
verify_program_scraping()


OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
